# Prolog

In [16]:
import os
import sys
import shutil
from IPython.display import display, Markdown, Latex

# Set the notebook name
docname = "tekstoppgaver"
nbname = f"{docname}.ipynb"
pdfname = f"{docname}.pdf"

cwd = os.path.realpath(".")
assert len(cwd) > 0 and cwd != "/"

# Initialize temporary figures directory
figsdir = "tmpfigs"
figsdir_full = f"{cwd}/{figsdir}"

if os.path.exists(figsdir):
    shutil.rmtree(figsdir)

os.mkdir(figsdir)

print(f"  > Current directory: {cwd}", file=sys.stderr)
print(f"  > Target notebook: {nbname}", file=sys.stderr)
print(f"  > Output file: {pdfname}", file=sys.stderr)
print(f"  > Figures directory: {figsdir_full}", file=sys.stderr)

# Clear the page
def clearpage() -> None:
    display(Latex(r"\clearpage"))

  > Current directory: /home/johnc/Projects/IN2060_oblig_3
  > Target notebook: tekstoppgaver.ipynb
  > Output file: tekstoppgaver.pdf
  > Figures directory: /home/johnc/Projects/IN2060_oblig_3/tmpfigs


# Del 2

1. I henhold til AAPCS32, så skal følgende heltallsregistere bevares av funksjonen:

   - `r4-r11` / `v1-v8` (delte verdier, osv.)
   - `r13` / `sp`
   
   Kalleren skal så bevare følgende heltallsregistere:

   - `r0-r3` / `a1-a4` (argumenter, returverdi)
   - `r12` / `ip`
   - `r14` / `lr`
   
   Hvis prosessoren støtter flyttallsregning, så skal

    - `s0-s15` / `d0-d7`/ `q0-q3` bevares av kalleren
    - `s16-s31` / `d8-d15` / `q4-q7` bevares av funksjonen

2. Funksjonen `fib` *skal* ta indekstallet i `r0` og bruke den til å regne ut resultatet. Dette *skal* returneres i `r0`. Argumentet er altså "brukt opp", og vi får resultatet i registeren som *for en stund siden* var argumentet.

3. For at dette programmet skal avslutte riktig, så måtte `main` lagre `lr` på starten, da den utfører en kall til `printf` -- altså at den ikke er en *bladfunksjon*. Et slik kall til en ekstern funksjon vil ellers "bruke opp" `lr`, sånn at programmet *ikke* klarer å gi tilbake kontrollen til operativsystemet.

In [17]:
clearpage()

<IPython.core.display.Latex object>

# Del 3

1. Den største forskjellen, er antallet *direktiver* som brukes. I den håndskrevne versjonen, så er antall direktiver minimal: det erklæres en tekst-seksjon (kjørbar kode) og funksjonen `main` utpekes som *inngangspunkt* ("entry point"). I den genererte versjonen, så er instruksene og direktivene annerledes: rekkefølgen på trinnene er rokket om for størrelse og effektivitet, metadata lagres i direktivene, osv.
2. Det er ikke noe vesentlig forskjell mellom `-Os`, `-O2` og `-O3`, da selveste logikken er verken minne-intensiv eller gren-intensiv ("branch-heavy"). Forskjellene ligger hovedsakelig i direktivene, da størrelsen til programmet kan endre seg noe mellom variantene.
3. Diskusjonen dreier seg hovedsakelig om skillen mellom *reglement* ("policy") og *mekanisme* ("mechanism").

   - Det største argumentet *for* å bruke en kompilator framfor en assembler, er *portabilitet*: et kompilert språk formidler hovedsakelig *reglement*, mens assemblerspråk formidler hovedsakelig *mekanisme*: assemblerspråk er altså maskinspråk med forståelige navnelapper ("mnemonics").
   - På den andre siden, finnes det visse rutiner som *på lovlig vis* ikke kan implementeres i kompilerte språk (I/O, oppstartsrutiner, osv.) I slike tilfeller, er mekanisme den eneste fornuftige fortolkningen -- noe assembler er egnet for.

# Del 4
## Bakgrunn
Flyttall som følger standarden **IEEE 754** *skal* kunne representeres på formen
$$
    x = (-1)^{s}\cdot\left(1 + \frac{f}{2^M}\right)\cdot 2^{e - b}
$$
der $s$ er fortegnsbit-en, $f$ er *bitfeltet* med lengde $M$ som inneholder brøkdelen, $e$ er bitfeltet som inneholder eksponenten *med forskyvning* ("biased exponent") og *b* er forskyvningen. Dette kan formuleres på en lignende måte:
$$
    x = (-1)^{s} \cdot (1.f)_2 \cdot 2^{e-b}
$$
Addisjon kan så representeres slik:
$$
    (1.a)_2 \cdot 2^{n} + (1.b)_2 \cdot 2^{m} = \left[(cd.e)_2 \cdot 2^{-c} \right] \cdot 2^{n + c}
$$
der $n,m: n \geq m$ er eksponentene, $a$, $b$ og $e$ er bitfelter med lengde $M$, og $c,d \in \{0, 1\}$.

Bitfeltet for $x$ er så gitt ved
$$
    X_{31:0} = s \cdot 2^{N-1} + e \cdot 2^{M} + f \equiv \mathtt{(s \ll (N-1))\, |\, (e \ll M)\, |\, f} 
$$
For 32-bits flyttall, så er $M = 23$ og $b = 127$.


In [19]:
clearpage()

<IPython.core.display.Latex object>

## Besvarelse

1. Vi har $2.0_{10} = (1.000\dots)_2 \cdot 2^{1} \equiv (1.000\dots)_2 \cdot 2^{128 - 127}$, sånn at $f = 0 \equiv \mathtt{0x00}$ og $e = 128 \equiv \mathtt{0x80}$. Bitfeltet for $2.0_{10}$ er så $$X_{31:0} = \mathtt{(0x80 \ll 23) \, |\, 0x00} = \mathtt{0x40000000}$$
2. Vi har $3.0_{10} = (1.100\dots)_2 \cdot 2^{1} \equiv (1.100\dots)_2 \cdot 2^{128 - 127}$, sånn at $f = \mathtt{1 \ll 22} \equiv \mathtt{0x400000}$ og $e = 128 \equiv \mathtt{0x80}$. Bitfeltet for $3.0_{10}$ er så $$X_{31:0} = \mathtt{(0x80 \ll 23)\, |\, 0x400000} = \mathtt{0x40400000}$$
3. Vi må først normalisere tallet: $0.50390625_{10} = 1.0078125_{10} \cdot 2^{-1} \equiv (1.0000001000\dots)_{2} \cdot 2^{-1}$. Vi har så $f = \mathtt{0x10000}$ og $e = 126 \equiv \mathtt{0x7E}$, sånn at $$X_{31:0} = \mathtt{(0x7E \ll 23)\, |\, 0x10000} = \mathtt{0x3f010000}$$
4. Vi har \begin{align*}
    2.0_{10} + 0.50390625_{10} &= (1.000\dots)_{2} \cdot 2^{1} + (1.0000001000\dots)_{2} \cdot 2^{-1} \\
    &= (1.000\dots)_{2} \cdot 2^{1} + (0.010000001000\dots)_{2} \cdot 2^{1} \\
    &= (1.010000001000\dots)_{2} \cdot 2^{0}
    \end{align*}
   sånn at $e = 128 = \mathtt{0x80}$ og $f = \mathtt{(1 \ll 21)\, |\, (1 \ll 14)} = \mathtt{0x204000}$. Bitfeltet til denne summen er så
   $$
       X_{31:0} = \mathtt{(0x80 \ll 23)\, |\, 0x204000} = \mathtt{0x40204000}
   $$

# Epilog

In [21]:
# The question is: can a cell hide itself
print(f" I: Exporting notebook as PDF... (output file: {pdfname})", file=sys.stderr)
print(f" I: Current path: {os.path.realpath('.')}", file=sys.stderr)
_ = os.system(f'\
jupyter nbconvert \
--to pdf \
--TagRemovePreprocessor.remove_cell_tags="hide-cell" \
--TagRemovePreprocessor.remove_input_tags="hide-input" \
{nbname}')

# --- Nothing that is hidden, can be shown beyond this point --- #

 I: Exporting notebook as PDF... (output file: tekstoppgaver.pdf)
 I: Current path: /home/johnc/Projects/IN2060_oblig_3
[NbConvertApp] Converting notebook tekstoppgaver.ipynb to pdf
[NbConvertApp] Writing 26145 bytes to notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', 'notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', 'notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 41139 bytes to tekstoppgaver.pdf
